In [ ]:
# default_exp hirise

# HiRISE

> Data management for HiRISE.

Currently depending on my older `pyrise` module, which will be more cleanly implemented here over time.

In [ ]:
# hide
from nbverbose.showdoc import show_doc  # noqa

In [ ]:
# export

import warnings

import hvplot.xarray  # noqa
import rasterio
import requests
import rioxarray as rxr
from dask import compute, delayed
from fastcore.utils import Path
from planetarypy.config import config
from planetarypy.pds.apps import get_index
from planetarypy.utils import url_retrieve
from pyrise import products as prod
from yarl import URL

warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)

In [ ]:
# export
storage_root = config.storage_root / "missions/mro/hirise"
baseurl = URL("https://hirise-pds.lpl.arizona.edu/download/PDS")
rdrindex = get_index("mro.hirise", "rdr")

Stored index is up-to-date.


In [ ]:
# export
class OBSERVATION_ID:
    """Manage HiRISE observation ids.

    For example PSP_003092_0985.

    `phase` is set to PSP for orbits < 11000, no setting required.

    Parameters
    ----------
    obsid : str, optional
        One can optionally also create an 'empty' OBSERVATION_ID object and set the
        properties accordingly to create a new obsid.
    """

    def __init__(self, obsid=None):
        if obsid is not None:
            phase, orbit, targetcode = obsid.split("_")
            self._orbit = int(orbit)
            self._targetcode = targetcode
        else:
            self._orbit = None
            self._targetcode = None

    @property
    def orbit(self):
        return str(self._orbit).zfill(6)

    @orbit.setter
    def orbit(self, value):
        if value > 999999:
            raise ValueError("Orbit cannot be larger than 999999")
        elif len(value) != 6:
            raise ValueError("Orbit string must be 6 digits.")
        self._orbit = value

    @property
    def targetcode(self):
        return self._targetcode

    @targetcode.setter
    def targetcode(self, value):
        if len(str(value)) != 4:
            raise ValueError("Targetcode must be exactly 4 characters.")
        self._targetcode = value

    @property
    def phase(self):
        return "PSP" if int(self.orbit) < 11000 else "ESP"

    def __str__(self):
        return "{}_{}_{}".format(self.phase, self.orbit, self.targetcode)

    def __repr__(self):
        return self.__str__()

    @property
    def s(self):
        return self.__str__()

    @property
    def upper_orbit_folder(self):
        """
        get the upper folder name where the given orbit folder is residing on the
        hisync server
        """
        lower = int(self.orbit) // 100 * 100
        return "_".join(["ORB", str(lower).zfill(6), str(lower + 99).zfill(6)])

    @property
    def storage_path_stem(self):
        s = "{phase}/{orbitfolder}/{obsid}".format(
            phase=self.phase, orbitfolder=self.upper_orbit_folder, obsid=self.s
        )
        return s

In [ ]:
obsid = OBSERVATION_ID("PSP_003092_0985")

In [ ]:
assert obsid.orbit == "003092"

In [ ]:
assert obsid.targetcode == "0985"

In [ ]:
assert obsid.phase == "PSP"

In [ ]:
assert obsid.upper_orbit_folder == 'ORB_003000_003099'

In [ ]:
obsid.storage_path_stem

'PSP/ORB_003000_003099/PSP_003092_0985'

In [ ]:
# export
def check_url_exists(url):
    response = requests.head(url)
    if response.status_code < 400:
        return True
    else:
        return False

In [ ]:
# export
class PRODUCT_ID:
    """Manage storage paths for HiRISE RDR products (also EXTRAS.)

    Attributes `jp2_path` and `label_path` get you the official RDR product,
    with `kind` steering if you get the COLOR or the RED product.
    All other properties go to the RDR/EXTRAS folder.

    Parameters
    ----------
    initstr : str, optional
        PRODUCT_ID string like PSP_003902_0985_RED

    Note
    ----
    The "PDS" part of the path is handled in the OBSERVATION_ID class.

    """

    kinds = ["RED", "BG", "IR", "COLOR", "IRB", "MIRB", "MRGB", "RGB", "RGB.NOMAP"]

    @classmethod
    def from_path(cls, path):
        path = Path(path)
        return cls(path.stem)

    def __init__(self, initstr=None):
        if initstr is not None:
            tokens = initstr.split("_")
            self._obsid = OBSERVATION_ID("_".join(tokens[:3]))
            try:
                self.kind = tokens[3]
            except IndexError:
                self._kind = None
        else:
            self._kind = None

    @property
    def obsid(self):
        return self._obsid

    @obsid.setter
    def obsid(self, value):
        self._obsid = OBSERVATION_ID(value)

    @property
    def kind(self):
        return self._kind

    @kind.setter
    def kind(self, value):
        if value not in self.kinds:
            raise ValueError(f"kind must be in {self.kinds}")
        self._kind = value

    def __str__(self):
        return f"{self.obsid}_{self.kind}"

    def __repr__(self):
        return self.__str__()

    @property
    def s(self):
        return self.__str__()

    @property
    def storage_stem(self):
        return f"{self.obsid.storage_path_stem}/{self.s}"

    @property
    def label_fname(self):
        return f"{self.s}.LBL"

    @property
    def label_path(self):
        return "RDR/" + self.storage_stem + ".LBL"

    def _make_url(self, obj):
        path = getattr(self, f"{obj}_path")
        url = baseurl / str(path)
        if not check_url_exists(url):
            warnings.warn(f"{url} does not exist on the server.")
        return url

    def __getattr__(self, item):
        tokens = item.split("_")
        try:
            if tokens[-1] == "url":
                return self._make_url("_".join(tokens[:-1]))
        except IndexError:
            raise ValueError(f"No attribute named '{item}' found.")

    @property
    def jp2_fname(self):
        return self.s + ".JP2"

    @property
    def jp2_path(self):
        prefix = "RDR/"
        postfix = ""
        if self.kind not in ["RED", "COLOR"]:
            prefix += "EXTRAS/"
        if self.kind in ["IRB"]:
            postfix = ".NOMAP"
        return prefix + self.storage_stem + postfix + ".JP2"

    @property
    def nomap_jp2_path(self):
        if self.kind in ["RED", "IRB", "RGB"]:
            return f"EXTRAS/RDR/{self.storage_stem}.NOMAP.JP2"
        else:
            raise AttributeError(f"No NOMAP exists for {self.kind}.")

    @property
    def quicklook_path(self):
        if self.kind in ["COLOR", "RED"]:
            return Path("EXTRAS/RDR/") / (self.storage_stem + ".QLOOK.JP2")
        else:
            raise AttributeError(f"No quicklook exists for {self.kind} products.")

    @property
    def abrowse_path(self):
        if self.kind in ["COLOR", "MIRB", "MRGB", "RED"]:
            return Path("EXTRAS/RDR/") / (self.storage_stem + ".abrowse.jpg")
        else:
            raise AttributeError(f"No abrowse exists for {self.kind}")

    @property
    def browse_path(self):
        inset = ""
        if self.kind in ["IRB", "RGB"]:
            inset = ".NOMAP"
        if self.kind not in ["COLOR", "MIRB", "MRGB", "RED", "IRB", "RGB"]:
            raise AttributeError(f"No browse exists for {self.kind}")
        else:
            return Path("EXTRAS/RDR/") / (self.storage_stem + inset + ".browse.jpg")

    @property
    def thumbnail_path(self):
        if self.kind in ["BG", "IR"]:
            raise AttributeError(f"No thumbnail exists for {self.kind}")
        inset = ""
        if self.kind in ["IRB", "RGB"]:
            inset = ".NOMAP"
        return Path("EXTRAS/RDR/") / (self.storage_stem + inset + ".thumb.jpg")

    @property
    def nomap_thumbnail_path(self):
        if self.kind in ["RED", "IRB", "RGB"]:
            return Path("EXTRAS/RDR") / (self.storage_stem + ".NOMAP.thumb.jpg")
        else:
            raise AttributeError(f"No NOMAP thumbnail exists for {self.kind}")

    @property
    def nomap_browse_path(self):
        if self.kind in ["RED", "IRB", "RGB"]:
            return Path("EXTRAS/RDR") / (self.storage_stem + ".NOMAP.browse.jpg")

    @property
    def edr_storage_stem(self):
        return "EDR/" + self.storage_stem

In [ ]:
pid = PRODUCT_ID('ESP_056531_0940_RED')

In [ ]:
assert isinstance(pid.obsid, OBSERVATION_ID)

In [ ]:
assert pid.kind == "RED"

In [ ]:
assert pid.label_fname == 'ESP_056531_0940_RED.LBL'

In [ ]:
pid.label_path

'RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RED.LBL'

### Paths and URLS
Each `xxx_path` attribute is also available as `xxx_url`, and accessing the `_url` parameter immediately checks for its existence on the server.

In [ ]:
import inspect


def isprop(v):
    return isinstance(v, property)


def get_properties(classname, substring):
    names = [
        name
        for (name, value) in inspect.getmembers(classname, isprop)
        if name.endswith(substring)
    ]
    return names

In [ ]:
get_properties(PRODUCT_ID, "_path")

['abrowse_path',
 'browse_path',
 'jp2_path',
 'label_path',
 'nomap_browse_path',
 'nomap_jp2_path',
 'nomap_thumbnail_path',
 'quicklook_path',
 'thumbnail_path']

In [ ]:
for prop in get_properties(PRODUCT_ID, "_path"):
    print(prop)
    print(getattr(pid, prop))
    urlattr = prop.replace("_path", "_url")
    print(urlattr)
    url = getattr(pid, urlattr)
    print(url)

abrowse_path
EXTRAS/RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RED.abrowse.jpg
abrowse_url
https://hirise-pds.lpl.arizona.edu/download/PDS/EXTRAS/RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RED.abrowse.jpg
browse_path
EXTRAS/RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RED.browse.jpg
browse_url
https://hirise-pds.lpl.arizona.edu/download/PDS/EXTRAS/RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RED.browse.jpg
jp2_path
RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RED.JP2
jp2_url
https://hirise-pds.lpl.arizona.edu/download/PDS/RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RED.JP2
label_path
RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RED.LBL
label_url
https://hirise-pds.lpl.arizona.edu/download/PDS/RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RED.LBL
nomap_browse_path
EXTRAS/RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RED.NOMAP.browse.jpg
nomap_browse_url
https://hirise-

In [ ]:
pid.storage_stem

'ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RED'

In [ ]:
pid.nomap_browse_url

URL('https://hirise-pds.lpl.arizona.edu/download/PDS/EXTRAS/RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RED.NOMAP.browse.jpg')

In [ ]:
pid = PRODUCT_ID('ESP_056531_0941_RED')

In [ ]:
pid.abrowse_url

/tmp/ipykernel_202233/988536391.py:82: UserWarning: https://hirise-pds.lpl.arizona.edu/download/PDS/EXTRAS/RDR/ESP/ORB_056500_056599/ESP_056531_0941/ESP_056531_0941_RED.abrowse.jpg does not exist on the server.
  warnings.warn(f"{url} does not exist on the server.")
/tmp/ipykernel_202233/988536391.py:82: UserWarning: https://hirise-pds.lpl.arizona.edu/download/PDS/EXTRAS/RDR/ESP/ORB_056500_056599/ESP_056531_0941/ESP_056531_0941_RED.abrowse.jpg does not exist on the server.
  warnings.warn(f"{url} does not exist on the server.")


URL('https://hirise-pds.lpl.arizona.edu/download/PDS/EXTRAS/RDR/ESP/ORB_056500_056599/ESP_056531_0941/ESP_056531_0941_RED.abrowse.jpg')

In [ ]:
# export
class RGB_NOMAP:
    def __init__(self, obsid):
        self.obsid = obsid
        if self.local_path.exists():
            self.read()  # this is fine, as it's using dask chunks, cheap op

    @property
    def product_id(self):
        return self.obsid + "_COLOR"

    @property
    def filename(self):
        return self.obsid + ".JP2"

    @property
    def pid(self):
        pid = prod.PRODUCT_ID(self.obsid)
        pid.kind = "RGB"
        return pid

    @property
    def meta(self):
        s = rdrindex.query("PRODUCT_ID == @self.product_id").squeeze()
        s.index = s.index.str.lower()
        return s

    # several things in the PDS path either have changed or I did it wrong in pyrise
    @property
    def nomap_jp2_path(self):
        p = Path("EXTRAS/") / self.pid.nomap_jp2_path.replace("/EXTRAS", "")
        return p

    @property
    def url(self):
        return baseurl / str(self.nomap_jp2_path)

    @property
    def local_path(self):
        full = self.nomap_jp2_path
        return storage_root / (f"EXTRAS/RDR/{full.parent.name}/{full.name}")

    def download(self, overwrite=False):
        self.local_path.parent.mkdir(parents=True, exist_ok=True)
        if self.local_path.exists() and not overwrite:
            print("File exists. Use `overwrite=True` to download fresh.")
            return
        url_retrieve(self.url, self.local_path)

    def read(self):
        self.da = rxr.open_rasterio(self.local_path, chunks=(1, 2024, 2024))
        return self.da

    def plot_da(self, xslice=None, yslice=None):
        if xslice is not None or yslice is not None:
            data = self.da.isel(x=xslice, y=yslice)
        else:
            data = self.da

        return data.hvplot.image(
            x="x",
            y="y",
            rasterize=True,
            widget_location="top_left",
            cmap="gray",
            frame_height=800,
            frame_width=800,
            flip_yaxis=True,
        )

In [ ]:
obsid = "ESP_056531_0940"

In [ ]:
rgb = RGB_NOMAP(obsid)

In [ ]:
rgb.url

URL('https://hirise-pds.lpl.arizona.edu/download/PDS/EXTRAS/RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RGB.NOMAP.JP2')

In [ ]:
rgb.download()

File exists. Use `overwrite=True` to download fresh.


In [ ]:
rgb.read()

<xarray.DataArray (band: 3, y: 110000, x: 2024)>
dask.array<chunksize=(1, 2024, 2024), meta=np.ndarray>
Coordinates:
  * band     (band) int64 1 2 3
  * y        (y) float64 0.5 1.5 2.5 3.5 4.5 ... 1.1e+05 1.1e+05 1.1e+05 1.1e+05
  * x        (x) float64 0.5 1.5 2.5 3.5 ... 2.022e+03 2.022e+03 2.024e+03
Attributes:
    transform:   (1.0, 0.0, 0.0, 0.0, 1.0, 0.0)
    res:         (1.0, -1.0)
    is_tiled:    1
    nodatavals:  (nan, nan, nan)
    scales:      (1.0, 1.0, 1.0)
    offsets:     (0.0, 0.0, 0.0)

In [ ]:
rgb.plot_da(slice(0, 800), slice(0, 600))

Column
    [0] Row
        [0] WidgetBox
            [0] DiscreteSlider(formatter='%d', margin=(20, 20, 20, 20), name='band', options=OrderedDict([('1', ...]), value=1, width=250)
        [1] HSpacer()
    [1] HoloViews(DynamicMap, widget_location='top_left')

In [ ]:
rgb.nomap_jp2_path

Path('EXTRAS/RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RGB.NOMAP.JP2')

In [ ]:
rgb.local_path

Path('/home/maye/big_drive/planetary_data/missions/mro/hirise/EXTRAS/RDR/ESP_056531_0940/ESP_056531_0940_RGB.NOMAP.JP2')

In [ ]:
rgb.download()

File exists. Use `overwrite=True` to download fresh.


In [ ]:
da = rgb.read()
da

<xarray.DataArray (band: 3, y: 110000, x: 2024)>
dask.array<chunksize=(1, 2024, 2024), meta=np.ndarray>
Coordinates:
  * band     (band) int64 1 2 3
  * y        (y) float64 0.5 1.5 2.5 3.5 4.5 ... 1.1e+05 1.1e+05 1.1e+05 1.1e+05
  * x        (x) float64 0.5 1.5 2.5 3.5 ... 2.022e+03 2.022e+03 2.024e+03
Attributes:
    transform:   (1.0, 0.0, 0.0, 0.0, 1.0, 0.0)
    res:         (1.0, -1.0)
    is_tiled:    1
    nodatavals:  (nan, nan, nan)
    scales:      (1.0, 1.0, 1.0)
    offsets:     (0.0, 0.0, 0.0)

In [ ]:
rgb.meta

volume_id                                                              MROHR_0001
file_name_specification         RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_...
instrument_host_id                                                            MRO
instrument_id                                                              HIRISE
observation_id                                                    ESP_056531_0940
product_id                                                  ESP_056531_0940_COLOR
product_version_id                                                              1
target_name                                                                  MARS
orbit_number                                                                56531
mission_phase_name                                         Extended Science Phase
rationale_desc                  Region dubbed Manhattan classic araneiform ter...
observation_start_time                                        2018-08-18 09:22:35
observation_star

In [ ]:
rgb.url

URL('https://hirise-pds.lpl.arizona.edu/download/PDS/EXTRAS/RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RGB.NOMAP.JP2')

In [ ]:
rgb.nomap_jp2_path

Path('EXTRAS/RDR/ESP/ORB_056500_056599/ESP_056531_0940/ESP_056531_0940_RGB.NOMAP.JP2')

In [ ]:
# export


class RGB_NOMAPCollection:
    """Class to deal with a set of RGB_NOMAP products."""

    def __init__(self, obsids):
        self.obsids = obsids

    def get_urls(self):
        """Get URLs for list of obsids.

        Returns
        -------
        List[yarl.URL]
            List of URL objects with the respective PDS URL for download.
        """
        urls = []
        for obsid in self.obsids:
            rgb = RGB_NOMAP(obsid)
            urls.append(rgb.url)
        self.urls = urls
        return urls

    @property
    def local_paths(self):
        paths = []
        for obsid in self.obsids:
            rgb = RGB_NOMAP(obsid)
            paths.append(rgb.local_path)
        return paths

    def download_collection(self):
        lazys = []
        for obsid in self.obsids:
            rgb = RGB_NOMAP(obsid)
            lazys.append(delayed(rgb.download)())
        print("Launching parallel download...")
        compute(*lazys)
        print("Done.")